In [ ]:
import pandas as pd
import numpy as np
import re

df_list = [pd.read_csv(file, names = ['title','release_year','runtime','rating','id','release_date', 'director', 'star_actors', 'budget','gross_us_can','gross_world','user_reviews','star_rating','vote_count']) for file in ['movie_data.csv']]
df = pd.concat(df_list, ignore_index=True)
# df

#Format titles
df['title'] = df['title'].str.replace(r'[^a-zA-Z0-9_\s]', '', regex=True).str.replace(' ', '_')
df['title'] = df['title'].str.lower()

#Drop wins_nominations and star_actors because it is not scraped correctly
#Remove budget because there are too many null values
df = df.drop(columns=['budget', 'star_actors'])

#Drop all N/A and NaN values
df.replace('N/A', np.nan, inplace=True)
df.dropna(inplace = True)

#Format budget and box office data
def clean_currency(value):
    if isinstance(value, str):
        value = value.replace('$', '').replace(',', '').replace('(estimated)', '').strip()
    return pd.to_numeric(value, errors='coerce')

# df['budget'] = df['budget'].apply(clean_currency)
df['gross_us_can'] = df['gross_us_can'].apply(clean_currency)
df['gross_world'] = df['gross_world'].apply(clean_currency)

#Format IMDB popularity data
def clean_popularity(value):
    if isinstance(value, str):
        if 'K' in value:
            return float(value.replace('K', '')) * 1000
        elif 'M' in value:
            return float(value.replace('M', '')) * 1000000
    return pd.to_numeric(value, errors='coerce')

df['user_reviews'] = df['user_reviews'].apply(clean_popularity)
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')
df['vote_count'] = df['vote_count'].apply(clean_popularity)

#Calculate gross revenue
df.dropna(inplace = True)
df['gross_revenue (million USD)'] = (df['gross_us_can'] + df['gross_world'])/1000000
df = df.drop(columns = ['gross_us_can', 'gross_world'])

#Indicator variables for seasonality

def extract_month(date_str):
    return date_str.split()[0]

# Apply the function to create a release_month column
df['release_month'] = df['release_date'].apply(extract_month)

# Define a mapping from month names to seasons
month_to_season = {
    'January': 'winter', 'February': 'winter', 'March': 'spring',
    'April': 'spring', 'May': 'spring', 'June': 'summer',
    'July': 'summer', 'August': 'summer', 'September': 'fall',
    'October': 'fall', 'November': 'fall', 'December': 'winter'
}

# Map the release_month to seasons
df['season'] = df['release_month'].map(month_to_season)

# Create indicator variables for each season
df['spring'] = (df['season'] == 'spring').astype(int)
df['summer'] = (df['season'] == 'summer').astype(int)
df['fall'] = (df['season'] == 'fall').astype(int)
df['winter'] = (df['season'] == 'winter').astype(int)

# Drop the original release_date and season columns if needed
df.drop(columns=['release_date', 'season'], inplace=True)

data = pd.read_csv('popular_directors.csv')
director_list = data['Popular Directors'].tolist()
print(director_list)
df['is_popular_director'] = df['director'].apply(lambda x: 1 if x in director_list else 0)

#Format MPA rating
def clean_ratings(x):
    if x == "TV-MA" or x == "NC-17":
        return "R"
    elif x == "TV-14" or x == "PG-13":
        return "PG-13"
    elif x == "TV-PG" or x == "GP":
        return "PG"
    elif x == 'Not Rated' or x == 'Unrated' or x == "Approved":
        return "Unrated"
    else:
        return x

df["rating"] = df["rating"].apply(clean_ratings)

# Construct indicator variables for each MPA rating class
dummies = pd.get_dummies(df['rating'])
dummies = dummies[['G', 'PG', 'PG-13', 'R', 'Unrated']]
df = pd.concat([df, dummies], axis=1)
df = df.drop(columns = ['rating'])
df[['runtime', 'user_reviews', 'vote_count','G', 'PG', 'PG-13', 'R', 'Unrated']] = df[['runtime', 'user_reviews', 'vote_count','G', 'PG', 'PG-13', 'R', 'Unrated']].astype(int)
df

df.to_csv('imdb_movie_data.csv', index = False)